In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdummysubjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdummysubjectconsent 2 covering 2020-07-01 UTC to 2025

In [19]:

from edc_visit_schedule.models import SubjectScheduleHistory
from flourish_caregiver.models.signals import onschedule_helper_cls
from edc_appointment.models import Appointment
from flourish_caregiver.models import MaternalVisit

child_1_visit = MaternalVisit.objects.get(
    subject_identifier='B142-040990804-2', visit_code='2000M',
    schedule_name='a_enrol1_schedule1')

visit = MaternalVisit.objects.get(
    subject_identifier='B142-040990804-2', visit_code='2000M',
    schedule_name='b_sec1_schedule1')

visit.report_datetime = child_1_visit.report_datetime
visit.save()

visit_codes = [
    '2001M', '2002M', '2003M', '2004M', '2005M', '2006M', '2007M'
]

for visit_code in visit_codes:
    appointment = Appointment.objects.get(
        subject_identifier='B142-040990804-2', visit_code='2001M',
        schedule_name='b_sec_quart1_schedule1')
    appointment.delete()

apt = visit.appointment
apt.appt_datetime = child_1_visit.appointment.appt_datetime
apt.save()

onschedule_model = django_apps.get_model(
    apt.schedule.onschedule_model)

cohort_list = visit.schedule_name.split('_')

caregiver_visit_count = cohort_list[1][-1:]

cohort = '_'.join(['cohort', cohort_list[0], 'sec_quart'])

onschedule_obj = onschedule_model.objects.get(
    subject_identifier=visit.subject_identifier,
    schedule_name=apt.schedule_name)

base_appt_datetime = visit.report_datetime.replace(microsecond=0)
helper_cls = onschedule_helper_cls(visit.subject_identifier, )
schedule, onschedule_model_cls, schedule_name = helper_cls.get_onschedule_model(
    cohort=cohort,
    caregiver_visit_count=caregiver_visit_count,
    instance=visit)

schedule.put_on_schedule(
    subject_identifier=visit.subject_identifier,
    onschedule_datetime=base_appt_datetime,
    schedule_name=schedule_name,
    base_appt_datetime=base_appt_datetime)

onschedule_obj = schedule.onschedule_model_cls.objects.get(
    subject_identifier=visit.subject_identifier,
    schedule_name=schedule_name,
    child_subject_identifier='')

onschedule_obj.child_subject_identifier = onschedule_obj.child_subject_identifier
onschedule_obj.save()

sub_history_obj = SubjectScheduleHistory.objects.get(
    subject_identifier=visit.subject_identifier, visit_schedule_name='b1_visit_schedule1',
    schedule_name='b_sec_quart1_schedule1')

sub_history_obj.onschedule_datetime = onschedule_obj.onschedule_datetime
sub_history_obj.save()

